In [17]:
%reload_ext autoreload
%autoreload 2


In [27]:
import numpy as np
from scipy import stats
import pandas as pd
import math
import os
from datetime import date

import sys

from Classes import DataO
from Classes import MapO




In [22]:
#set paths for raw data retreaval and result storing

today = date.today()

# dd/mm/YY
d1 = today.strftime("%d%m%Y")

datapath="../3_Raw_Data/"
resultpath="../6_Results/"+d1+"_Maps/"

In [7]:
'TODO:plot spatial distribution of values, 3D histogram?'

'TODO:plot spatial distribution of values, 3D histogram?'

In [34]:
#### RUNNING ANALYSIS ####
#** Generating the threshold table ***

thresh_df=pd.DataFrame(
    columns=["Maxtemp",
            "Mintemp",
            "Avgtemp",
            "Rain",
            "Wind",
            "Snow"],
    index= ["Thermoelectric",
            "Hydroelectric",
            "Wind",
            "Solar PV",
            "Transformers",
            "Poles",
            "Overhead Lines",
            "Demand"]
)

thresh_df['Maxtemp']=[30,35,35,35,35,35,35,35]
thresh_df['Mintemp']=[3,0,0,0,0,0,0,0]
thresh_df['Avgtemp']=[20,25,25,25,25,25,25,25]
thresh_df['Rain']=[5,50,50,50,50,50,50,50]
thresh_df['Wind']=[20,30,30,30,30,30,30,30]
thresh_df['Snow']=[1,50,50,50,50,50,50,50]

'TESTING'
print(thresh_df)

                Maxtemp  Mintemp  Avgtemp  Rain  Wind  Snow
Thermoelectric       30        3       20     5    20     1
Hydroelectric        35        0       25    50    30    50
Wind                 35        0       25    50    30    50
Solar PV             35        0       25    50    30    50
Transformers         35        0       25    50    30    50
Poles                35        0       25    50    30    50
Overhead Lines       35        0       25    50    30    50
Demand               35        0       25    50    30    50


In [44]:
thresh_df.loc["Transformers",:]

correl_df = pd.DataFrame(
    columns=["Maxtemp",
            "Mintemp",
            "Avgtemp",
            "Rain",
            "Wind",
            "Snow"],
    index= ["Thermoelectric",
            "Hydroelectric",
            "Wind",
            "Solar PV",
            "Transformers",
            "Poles",
            "Overhead Lines",
            "Demand"]
)

correl_df.loc["Solar PV","Maxtemp"]= lambda x : -0.48*x

In [31]:
'MONTHLY AVERAGE DATA'

### LOAD ALL WEATHER VARIABLE DATA ###

#setting resolutions (for info only)
spatial_res=5
time_res='monthly'
start_year=2021
finish_year=2041 #finish year is included

#creating result directory
if not os.path.exists(resultpath):
    os.mkdir(resultpath)

#all data objects are created with all available data members in directory (fdpath)
#initialising runs data loading and coordinate vectors loading

Maxtemp=DataO.DataObject(title="5_Monthly_Maxtemp_2021_2040_UK",
                        varID="Maxtemp",sres=spatial_res,tres=time_res,syear=start_year,fyear=finish_year,
                        fdpath="../3_Raw_Data/5_Monthly_Maxtemp_2021_2040_UK",
                        respath=resultpath)


Avgtemp=DataO.DataObject(title="5_Monthly_Avgtemp_2021_2040_UK",
                        varID="Avgtemp",sres=spatial_res,tres=time_res,syear=start_year,fyear=finish_year,
                        fdpath="../3_Raw_Data/5_Monthly_Avgtemp_2021_2040_UK",
                        respath=resultpath)

Mintemp=DataO.DataObject(title="5_Monthly_Mintemp_2021_2040_UK",
                        varID="Mintemp",sres=spatial_res,tres=time_res,syear=start_year,fyear=finish_year,
                        fdpath="../3_Raw_Data/5_Monthly_Mintemp_2021_2040_UK",
                        respath=resultpath)

Rain=DataO.DataObject(title="5_Monthly_Rain_2021_2040_UK",
                        varID="Rain",sres=spatial_res,tres=time_res,syear=start_year,fyear=finish_year,
                        fdpath="../3_Raw_Data/5_Monthly_Rain_2021_2040_UK",
                        respath=resultpath)

Snow=DataO.DataObject(title="5_Monthly_Snow_2021_2040_UK",
                        varID="Snow",sres=spatial_res,tres=time_res,syear=start_year,fyear=finish_year,
                        fdpath="../3_Raw_Data/5_Monthly_Snow_2021_2040_UK",
                        respath=resultpath)

WeatherVariables=[Maxtemp,Mintemp,Avgtemp,Rain,Snow]

'TESTING'
for object in WeatherVariables:
    assert object.untreated_array.shape == (12,239,243,153), 'Loaded data array does not have expected shape'


In [45]:
### RUNNING STAT TESTS ###

Maxtemp.run_stats(KStest=False,stats=True,tp_90=True)
Mintemp.run_stats(KStest=False,stats=True,tp_90=False,tp_10=True)
Avgtemp.run_stats(KStest=False,stats=True,tp_90=True)
Rain.run_stats(KStest=False,stats=True,tp_90=True)
Snow.run_stats(KStest=False,stats=True,tp_90=True)


In [29]:
### MAP 1 ###

AvgTemp_Map=MapO.MapObject('AvgTemp_Map',Avgtemp,resultpath)


In [13]:
'TODO: update with correct row as component'

### SETTING THRESHOLDS FOR EACH VARIABLE ###
for variable in WeatherVariables:
    threshold=thresh_df[variable.varID]["Thermoelectric"]
    variable.set_threshold(threshold)


'Testing'
for variable in WeatherVariables:
    assert variable.threshold==thresh_df[variable.varID]["Thermoelectric"]


In [14]:
### COUNTING EXCESS DAYS ###

Maxtemp.counter()
Mintemp.counter()
Avgtemp.counter()
Rain.counter()
Snow.counter()


In [15]:
### CREATING MAP ###

Map_5_Monthly_2021_2040=MapO.MapObject("Maxtemp - Rel Excess days",Maxtemp,respath=resultpath)

for variable in WeatherVariables:
    if not (variable.title in Map_5_Monthly_2021_2040.dnames):
        Map_5_Monthly_2021_2040.addDataObject(variable)

Map_5_Monthly_2021_2040.build_props()
Map_5_Monthly_2021_2040.geojson_write('allwv')

In [79]:
'DAILY DATA'

### LOAD ALL WEATHER VARIABLE DATA ###


#setting resolutions

spatial_res=60
time_res='daily'
start_year=2021
finish_year=2041 #finish year is included

datapath="../3_Raw_Data/Masterina/All"

#creating result directory
if not os.path.exists(resultpath):
    os.mkdir(resultpath)

#all data objects are created with all available data members in directory (fdpath)
#initialising runs data loading and coordinate vectors loading

Maxtemp=DataO.DataObject(title="60_Daily_Maxtemp_2021_2041_UK",
                        varID="Maxtemp",sres=spatial_res,tres=time_res,syear=start_year,fyear=finish_year,
                        members=12,
                        total_rows=7200,
                        fdpath=datapath,
                        respath=resultpath)

'TESTING'

assert Maxtemp.untreated_array.shape == (75,1,7200,12), 'Loaded data array does not have expected shape'

: 